#### Available scene list

In [1]:
import json

    
f = open('../scene.json', "r")
scene_json = json.load(f)

scene_list = list(scene_json.keys())
scene_dict = scene_json

#### Sample positions from trajectory

In [2]:
import os


def sample_position(s_idx, r_idx, n_sample):
    
    scene_id = scene_list[s_idx]
    
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    
    traj_pth = f"{ds_pth}/{scene_id}/{r_idx}/traj_sub.txt"
    traj_fp = open(traj_pth, encoding="utf-8")
    traj = traj_fp.readlines()
    l_traj = len(traj)
    print(f"Traj length {l_traj}, adjusted length {n_sample}")
    
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-panorama/"
    if not os.path.exists(f"{ds_pth}/{scene_id}/{r_idx}"):
        os.makedirs(f"{ds_pth}/{scene_id}/{r_idx}")
    pos_pth = f"{ds_pth}/{scene_id}/{r_idx}/{scene_id}.txt"
    pos_fp = open(pos_pth, "w", encoding="utf-8")
    
    if l_traj >= n_sample:
        # The trajectory length is greater than threshold
        print(f"Sample trajectory in area {r_idx}")
        step = l_traj // n_sample
        for idx in range(n_sample):
            
            pos = traj[idx * step]
            pos_fp.write(pos)
    else:
        # The trajectory length is less than threshold
        print(f"Copy trajectory in area {r_idx}")
        times = n_sample // l_traj
        threshold = n_sample % l_traj
        pass

In [3]:
s_idx = 1
for r_idx in range(5):
    sample_position(s_idx, r_idx, 200)

s_idx = 3
for r_idx in range(2):
    sample_position(s_idx, r_idx, 200)
    
s_idx = 4
for r_idx in range(6):
    sample_position(s_idx, r_idx, 200)
    
s_idx = 5
for r_idx in range(3):
    sample_position(s_idx, r_idx, 200)
    
s_idx = 6
for r_idx in range(6):
    sample_position(s_idx, r_idx, 200)

Traj length 629, adjusted length 200
Sample trajectory in area 0
Traj length 718, adjusted length 200
Sample trajectory in area 1
Traj length 478, adjusted length 200
Sample trajectory in area 2
Traj length 327, adjusted length 200
Sample trajectory in area 3
Traj length 476, adjusted length 200
Sample trajectory in area 4
Traj length 887, adjusted length 200
Sample trajectory in area 0
Traj length 458, adjusted length 200
Sample trajectory in area 1
Traj length 646, adjusted length 200
Sample trajectory in area 0
Traj length 303, adjusted length 200
Sample trajectory in area 1
Traj length 427, adjusted length 200
Sample trajectory in area 2
Traj length 733, adjusted length 200
Sample trajectory in area 3
Traj length 536, adjusted length 200
Sample trajectory in area 4
Traj length 624, adjusted length 200
Sample trajectory in area 5
Traj length 505, adjusted length 200
Sample trajectory in area 0
Traj length 382, adjusted length 200
Sample trajectory in area 1
Traj length 465, adjusted

#### Visualize sample positions

In [4]:
from PIL import Image
import matplotlib.pyplot as plt


def visualize_panorama_sp(s_idx, r_idx):
    
    scene_id = scene_list[s_idx]
    
    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)
    
    # Visualize map image
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    img_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    img = Image.open(img_pth).convert("RGB")
    plt.imshow(img)
    
    # Visualize sample position
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-panorama/"
    gt_pth = f"{ds_pth}/{scene_id}/{r_idx}/{scene_id}.txt"
    gt_positions = []

    gt_fp = open(gt_pth, encoding="utf-8")
    for gt_p in gt_fp.readlines():
        gt_p = ((gt_p).replace("\n", "")).split(",")
        gt_positions.append((gt_p[0], gt_p[1]))

    # Translate sample position to map coordinate
    scene_trans = scene_dict[scene_id]

    traj_x = []
    traj_y = []
    
    for gt_p in gt_positions:
        
        pos_x = float(gt_p[0]) * scene_trans[2] + scene_trans[0]
        pos_y = float(gt_p[1]) * scene_trans[2] + scene_trans[1]
        traj_x.append(pos_x)
        traj_y.append(pos_y)    
        
    plt.scatter(traj_x, traj_y)
    plt.savefig(f"{ds_pth}/{scene_id}/{r_idx}/sample_position.png")

In [5]:
s_idx = 1
for r_idx in range(5):
    visualize_panorama_sp(s_idx, r_idx)
    plt.cla()

s_idx = 3
for r_idx in range(2):
    visualize_panorama_sp(s_idx, r_idx)
    plt.cla()
    
s_idx = 4
for r_idx in range(6):
    visualize_panorama_sp(s_idx, r_idx)
    plt.cla()
    
s_idx = 5
for r_idx in range(3):
    visualize_panorama_sp(s_idx, r_idx)
    plt.cla()
    
s_idx = 6
for r_idx in range(6):
    visualize_panorama_sp(s_idx, r_idx)
    plt.cla()
    
plt.close("all")

C:\Users\12871\AppData\Local\Temp\ipykernel_21800\1528735549.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))


#### Sample data

In [6]:
import os
import yaml
import numpy as np
import random

from tqdm import tqdm

from utils import *

import igibson
from igibson.envs.igibson_env import iGibsonEnv
from igibson.render.profiler import Profiler


def sample_scene(s_idx, r_idx, n_sample=None):
    
    scene_id = scene_list[s_idx]
    
    # Read sample positions
    pos_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-panorama/"
    pos_pth = f"{pos_pth}/{scene_id}/{r_idx}/{scene_id}.txt"
    
    poses = []
    pos_fp = open(pos_pth, "r", encoding="utf-8")
    for pos_l in pos_fp:
        pos_l = pos_l.replace("\n", "")
        pos_l = pos_l.split(",")
        poses.append((float(pos_l[0]), float(pos_l[1]), 1.0, float(pos_l[2])))
    
    if n_sample is not None:
        assert n_sample < len(poses)
    else:
        n_sample = len(poses)

    # Sample data
    tgt_ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data/"
    tgt_pth = f"{tgt_ds_pth}/{scene_id}/{r_idx}/"
        
    config_filename = os.path.join(igibson.configs_path, "turtlebot_nav.yaml")
    config_data = yaml.load(open(config_filename, "r"), Loader=yaml.FullLoader)
    
    # 启动阴影和基于物理的渲染 (optional)
    config_data["enable_shadow"] = True
    config_data["enable_pbr"] = True
    config_data["output"] = ["rgb", "depth", "seg", "ins_seg", "normal"]
    
    config_data["load_object_categories"] = []  # Uncomment this line to accelerate loading with only the building
    config_data["visible_target"] = False
    config_data["visible_path"] = False
    # Set specific scene
    config_data["scene_id"] = scene_id
    
    env = iGibsonEnv(config_file=config_data, scene_id=scene_id, mode="gui_interactive")
    s = env.simulator

    # Set a better viewing direction
    # s.viewer.initial_pos = [-2, 1.4, 1.2]
    # s.viewer.initial_view_direction = [0.6, -0.8, 0.1]
    # s.viewer.reset_viewer()
    
    print(f"Resetting environment... scene {scene_id}, room {r_idx}")
    env.reset()
        
    # Sample data of each position
    for p_idx in tqdm(range(len(poses)), desc=f"{scene_id} {r_idx}"):
        
        env.reset()
        # print("==============================")
        # with Profiler("Environment action step"):
        state, reward, done, info = env.step([0.0, 0.0])
        
        position = poses[p_idx][:3]
        env.robots[0].set_position(position)
        
        # rotation_angle = random.uniform(0, 360)
        # rotation = euler_to_quaternion(0.0, 0.0, rotation_angle)
        # rotation = poses[p_idx][3]
        # rotation = euler_to_quaternion(0.0, 0.0, rotation)
        # env.robots[0].set_orientation(rotation)
        
        index = "{:04d}".format(p_idx)
        # sample_data(env, index, tgt_pth)
        
        if not os.path.exists(f"{tgt_pth}/{p_idx}/gt/"):
            os.makedirs(f"{tgt_pth}/{p_idx}/gt/")
        gt_f = open(f"{tgt_pth}/{p_idx}/gt/0000.txt", "w", encoding="utf-8")
        gt_f.write(f"{position[0]},{position[1]},{position[2]}")
        
        # rotate and sample data
        for idx, deg in enumerate(range(0, 360, 45)):
            q = euler_to_quaternion(0.0, 0.0, deg)
            env.robots[0].set_orientation(q)
            index = "{:04d}".format(0) + f"_{idx}"
            # 
            sample_data(env, index, f"{tgt_pth}/{p_idx}")
            
            # robot_position = env.robots[0].get_position()
            # print(robot_position)
            # robot_orientation = env.robots[0].get_orientation()
            # print(robot_orientation)


 _   _____  _  _
(_) / ____|(_)| |
 _ | |  __  _ | |__   ___   ___   _ __
| || | |_ || || '_ \ / __| / _ \ | '_ \
| || |__| || || |_) |\__ \| (_) || | | |
|_| \_____||_||_.__/ |___/ \___/ |_| |_|



In [7]:
s_idx = 1
for r_idx in range(5):
    sample_scene(s_idx, r_idx)

s_idx = 3
for r_idx in range(2):
    sample_scene(s_idx, r_idx)
    
s_idx = 4
for r_idx in range(6):
    sample_scene(s_idx, r_idx)
    
s_idx = 5
for r_idx in range(3):
    sample_scene(s_idx, r_idx)
    
s_idx = 6
for r_idx in range(6):
    sample_scene(s_idx, r_idx)

e:\anaconda3\envs\igibson\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Resetting environment... scene Beechwood_1_int, room 0


Beechwood_1_int 0: 100%|██████████| 200/200 [02:22<00:00,  1.40it/s]


Resetting environment... scene Beechwood_1_int, room 1


Beechwood_1_int 1: 100%|██████████| 200/200 [02:31<00:00,  1.32it/s]


...

Merom_1_int 3: 100%|██████████| 200/200 [02:12<00:00,  1.51it/s]


Resetting environment... scene Merom_1_int, room 4


Merom_1_int 4: 100%|██████████| 200/200 [02:05<00:00,  1.59it/s]


Resetting environment... scene Merom_1_int, room 5


Merom_1_int 5: 100%|██████████| 200/200 [02:11<00:00,  1.52it/s]


#### Preprocess: depth image

In [12]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm


def preprocess_depth(s_idx, r_idx):
    
    scene_id = scene_list[s_idx]
    
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data/"
    ds_dir = f"{ds_pth}/{scene_id}/{r_idx}/"
    
    sp_list = os.listdir(ds_dir)
    for sp in tqdm(sp_list):
        
        sp_dir = f"{ds_dir}/{sp}/"
        
        src_dir = f"{sp_dir}/d/"
        tgt_dir = f"{sp_dir}/d_processed/"
        if not os.path.exists(tgt_dir):
            os.makedirs(tgt_dir)
        
        for d_pth in os.listdir(src_dir):
            
            d = np.array(Image.open(f"{src_dir}/{d_pth}").convert("L"))
            d[d == 0] = 255
            d = Image.fromarray(d)
            
            d.save(f"{tgt_dir}/{d_pth}")

In [13]:
s_idx = 1
for r_idx in range(5):
    preprocess_depth(s_idx, r_idx)

s_idx = 3
for r_idx in range(2):
    preprocess_depth(s_idx, r_idx)
    
s_idx = 4
for r_idx in range(6):
    preprocess_depth(s_idx, r_idx)
    
s_idx = 5
for r_idx in range(3):
    preprocess_depth(s_idx, r_idx)
    
s_idx = 6
for r_idx in range(6):
    preprocess_depth(s_idx, r_idx)